In [1]:
import skimage
import os
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib
import matplotlib.pyplot as plt
from skimage import data
from skimage import io
import glob
from skimage.viewer import ImageViewer
import cv2
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn import metrics
%matplotlib inline
import seaborn
import numpy as np
from scipy.spatial.distance import cdist, pdist
from keras.preprocessing import image
import tqdm
import SceneDetection 

/anaconda3/lib/python3.6/site-packages/skimage/viewer/utils/core.py:10: UserWarning: Recommended matplotlib backend is `Agg` for full skimage.viewer functionality.
  warn("Recommended matplotlib backend is `Agg` for full "
Using TensorFlow backend.
/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#Define List of Users and define File-Path to Users Folder
user_list_path = '/Users/kmotwani/Desktop/Me/Education/Courses/Capstone Project/Scrape/final.txt'
with open(user_list_path, encoding="ISO-8859-1") as f:
    content = f.readlines()
list_users = [x.strip() for x in content] 
print("No# of Users Selected:",len(list_users))

path = "/Users/kmotwani/Desktop/Me/Education/Courses/Capstone Project/Scrape/"
model_path = "/Users/kmotwani/Desktop/Me - Local/Education/Courses/Capstone Project/keras-retinanet/snapshots/resnet50_coco_best.h5"
dest_path = "/Users/kmotwani/Desktop/Me - Local/Education/Courses/Capstone Project/Clustering_"

No# of Users Selected: 30


In [4]:
#Get True Caption Dataframes
brands = ['athenaprocopiou', 'daftcollectionofficial', 'dodobaror', 'emporiosirenuse', 
          'heidikleinswim', 'lisamariefernandez', 'loupcharmant', 'miguelinagambaccini',
          'muzungusisters', 'zeusndione']

final_dfs = []
for ind, brand in enumerate(brands):
    print(ind+1,"/",len(brands))
    temp_df = pd.read_csv('/Users/kmotwani/Dropbox/Harvard/Capstone_EmpSirenuse/Datasets/' + brand 
                          + '_users_captions.csv')
    temp_df['url_img'] = temp_df['url_img'].apply(lambda x: x.split('/')[-1])
    final_dfs.append(temp_df)
    del temp_df

#Concatenate Dataframe
df_fol_posts = pd.concat(final_dfs)
del final_dfs

#Drop Unnamed: 0
del df_fol_posts['Unnamed: 0']
display(df_fol_posts.head())

1 / 10
2 / 10
3 / 10
4 / 10
5 / 10
6 / 10
7 / 10
8 / 10
9 / 10
10 / 10


,id_post,username,url_img,link_post,caption,comment_count,likes_count,followers_count,brand
0,1639442937632026948,theuptowncharade,23101000_1936185076643469_7954762517228552192_...,https://www.instagram.com/p/BbAeKpnFHFE,#stylegram #woolcoat http://liketk.it/2tiqD #l...,19.0,386.0,22509,athenaprocopiou
1,1638712234560824875,theuptowncharade,23101054_125805054767734_5062654925922631680_n...,https://www.instagram.com/p/Ba94BhWFaYr,Cozy fall wear #stylegram #fall @liketoknow.it...,12.0,381.0,22509,athenaprocopiou
2,1637262979115677259,theuptowncharade,23098503_761974183996128_5060507081497378816_n...,https://www.instagram.com/p/Ba4ugFCFnZL,❤️❤️ #madforplaid (under $200!) #stylegram @li...,17.0,679.0,22509,athenaprocopiou
3,1633640806568326751,theuptowncharade,22793767_323240268144099_7918006895778988032_n...,https://www.instagram.com/p/Bar26h-FnZf,Classic new arrivals! #stylegram #newarrivals ...,15.0,321.0,22509,athenaprocopiou
4,1631529105433637961,theuptowncharade,22638851_1051035104998998_34452445237084160_n.jpg,https://www.instagram.com/p/BakWxPCFTRJ,Sunday funday! #nyc #batterypark,19.0,326.0,22509,athenaprocopiou


In [6]:
#Helper Function to get images from path
def get_images(path, list_users):
    #Get all Images of Users in the List
    user_imgs = {}
    for i in list_users:
        temp_path = path + i
        for j in glob.glob(temp_path + '/*.jpg'):
            file_name = j.replace(temp_path,'')[1:]
            img = image.load_img(j, target_size=(299, 299))
            user_imgs[(i,file_name)] = img
    return user_imgs 

#Get Images from User List and Path
user_imgs = get_images(path, list_users)
print("Number of images loaded:", len(user_imgs))

Number of images loaded: 6914


In [7]:
#Helper function to get image vector
def get_image_vector(data, obj):
    url, img_cap, img = data[0], data[1], np.array(data[1])
    r_mean, r_std = np.mean(img[:,:,0].ravel()), np.std(img[:,:,0].ravel())
    g_mean, g_std = np.mean(img[:,:,1].ravel()), np.std(img[:,:,1].ravel())
    b_mean, b_std = np.mean(img[:,:,2].ravel()), np.std(img[:,:,2].ravel())
    scene_caption = obj.predict(img_cap, False, 3)
    final = [int(r_mean), int(r_std), int(g_mean), int(g_std), int(b_mean), int(b_std), scene_caption]
    return final

In [16]:
#Helper function to convert image to d-dimension vector
def convert(user_imgs, obj, img_lim, seed):
    user_features = []
    np.random.seed(seed)
    final_imgs = list(user_imgs.items())
    for j in tqdm.tqdm(range(0,img_lim)):
        i = final_imgs[np.random.randint(len(final_imgs))]
        img_vec = get_image_vector((i[0][1], i[1]), obj)
        final_vec = [i[0][0], i[0][1]]
        temp = [final_vec.append(k) for k in img_vec]
        user_features.append(np.array(final_vec))
    return user_features
    
#Convert Images
obj = SceneDetection.Main('word2idx.p', 'idx2word.p', 'time_inceptionV3_1.5987_loss.h5')
user_features = convert(user_imgs, obj, 6000, 9001)
print("\nFeature Vectors Created.")

100%|██████████| 6000/6000 [1:54:15<00:00,  1.14s/it]


Feature Vectors Created.


In [17]:
#Create Dataframe of Features and Identifiers
df = pd.DataFrame(user_features)
columns = ["User_Handle","URL","R_Mean", "R_STD", "G_Mean", "G_STD", "B_Mean", "B_STD", "Scene_Caption"]
df.columns = columns
print("Main DF:\n")
display(df.head())

Main DF:



,User_Handle,URL,R_Mean,R_STD,G_Mean,G_STD,B_Mean,B_STD,Scene_Caption
0,victoria__archer,23967426_332438503897769_1423963949920944128_n...,148,81,142,72,132,70,a dog race the the the the dog is in the the l...
1,briannapearl,21911154_181038529108764_948650394697859072_n.jpg,101,73,74,55,63,44,A woman with a camera looks at the man with to...
2,organicallyashley,19436435_463794853980886_5191315537472258048_n...,141,65,131,62,123,66,Two children play in an inflatable pool .
3,justinelee425,22280439_1986068614998720_1337003648294584320_...,142,71,141,68,90,77,a man in a white dog is in the grass .
4,siguemiestilo,21042841_1297956693647492_7898386858105634816_...,167,74,157,77,144,80,Two men are climbing up a rocky cliff .


In [29]:
#Merge Dataframes
df_merge = df.copy(deep=True)
df_merge['url_img'] = df['URL']
df_merge = df_merge.merge(df_fol_posts, on='url_img', how='inner',suffixes=('',''))
display(df_merge.head())
print("Length of Merged Dataframe:", len(df_merge))

,User_Handle,URL,R_Mean,R_STD,G_Mean,G_STD,B_Mean,B_STD,Scene_Caption,url_img,id_post,username,link_post,caption,comment_count,likes_count,followers_count,brand
0,siguemiestilo,21042841_1297956693647492_7898386858105634816_...,167,74,157,77,144,80,Two men are climbing up a rocky cliff .,21042841_1297956693647492_7898386858105634816_...,1589869754347182648,siguemiestilo,https://www.instagram.com/p/BYQWh47AFo4,Pierrot Le Fou.Fotogramas originales de la rep...,2.0,581.0,61488,dodobaror
1,siguemiestilo,21042841_1297956693647492_7898386858105634816_...,167,74,157,77,144,80,Two men are climbing up a rocky cliff .,21042841_1297956693647492_7898386858105634816_...,1589869754347182648,siguemiestilo,https://www.instagram.com/p/BYQWh47AFo4,Pierrot Le Fou.Fotogramas originales de la rep...,2.0,581.0,61581,miguelinagambaccini
2,thirlby,20688179_1801351643489678_5347056622197800960_...,190,67,173,68,158,78,A woman is sitting on a the the small table an...,20688179_1801351643489678_5347056622197800960_...,1576965860106896817,thirlby,https://www.instagram.com/p/BXighg6lwWx,THE GUIDE: HOW TO BE A MINDFUL HOST | on #thet...,13.0,250.0,25121,loupcharmant
3,miishka,20184473_1488106757891866_1025121217956282368_...,141,78,134,81,133,84,Two girls dressed in costumes on a hill,20184473_1488106757891866_1025121217956282368_...,1562827838964749831,miishka,https://www.instagram.com/p/BWwR6OqldYH,That almost weekend feeling..,3.0,156.0,10019,dodobaror
4,miishka,20184473_1488106757891866_1025121217956282368_...,141,78,134,81,133,84,Two girls dressed in costumes on a hill,20184473_1488106757891866_1025121217956282368_...,1562827838964749831,miishka,https://www.instagram.com/p/BWwR6OqldYH,That almost weekend feeling..,3.0,156.0,10019,dodobaror


Length of Merged Dataframe: 2657


In [19]:
#Keep mentioned columns
keep_list = ['R_Mean', 'R_STD', 'G_Mean', 'G_STD', 'B_Mean', 'B_STD',
       'Scene_Caption', 'url_img', 'id_post', 'username', 'link_post',
       'caption']

for i in df_merge.columns:
    if i not in keep_list:
        del df_merge[i]
display(df_merge.head())

,R_Mean,R_STD,G_Mean,G_STD,B_Mean,B_STD,Scene_Caption,url_img,id_post,username,link_post,caption
0,167,74,157,77,144,80,Two men are climbing up a rocky cliff .,21042841_1297956693647492_7898386858105634816_...,1589869754347182648,siguemiestilo,https://www.instagram.com/p/BYQWh47AFo4,Pierrot Le Fou.Fotogramas originales de la rep...
1,167,74,157,77,144,80,Two men are climbing up a rocky cliff .,21042841_1297956693647492_7898386858105634816_...,1589869754347182648,siguemiestilo,https://www.instagram.com/p/BYQWh47AFo4,Pierrot Le Fou.Fotogramas originales de la rep...
2,190,67,173,68,158,78,A woman is sitting on a the the small table an...,20688179_1801351643489678_5347056622197800960_...,1576965860106896817,thirlby,https://www.instagram.com/p/BXighg6lwWx,THE GUIDE: HOW TO BE A MINDFUL HOST | on #thet...
3,141,78,134,81,133,84,Two girls dressed in costumes on a hill,20184473_1488106757891866_1025121217956282368_...,1562827838964749831,miishka,https://www.instagram.com/p/BWwR6OqldYH,That almost weekend feeling..
4,141,78,134,81,133,84,Two girls dressed in costumes on a hill,20184473_1488106757891866_1025121217956282368_...,1562827838964749831,miishka,https://www.instagram.com/p/BWwR6OqldYH,That almost weekend feeling..


In [24]:
#Drop Duplicates
df_merge = df_merge.drop_duplicates(subset=['url_img'], keep='first')
print("Merge DF Length :", len(df_merge))

Merge DF Length : 1453


In [25]:
#Save Dataframes
df.to_csv("/Users/kmotwani/Dropbox/Harvard/Capstone_EmpSirenuse/scene_df_total_6K.csv")
df_merge.to_csv("/Users/kmotwani/Dropbox/Harvard/Capstone_EmpSirenuse/scene_df_1453.csv")